In [ ]:
!pip install opencv-python ipywidgets matplotlib

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown


In [ ]:
# Upload an image from your system
uploaded = files.upload()
image_path = list(uploaded.keys())[0]
image = cv2.imread(image_path)


In [ ]:
# Show original image
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image_rgb)
plt.title("Original Image")
plt.axis('off')
plt.show()


In [ ]:
# Threshold sliders
low_th = widgets.IntSlider(value=50, min=0, max=255, step=1, description='Low Threshold')
high_th = widgets.IntSlider(value=150, min=0, max=255, step=1, description='High Threshold')

# Color selector
color_picker = widgets.Dropdown(
    options={
        'Blue': [255, 0, 0],
        'Green': [0, 255, 0],
        'Red': [0, 0, 255],
        'Yellow': [0, 255, 255],
        'Purple': [255, 0, 255],
        'White': [255, 255, 255]
    },
    value=[255, 255, 255],
    description='Edge Color'
)

# Button to apply changes
button = widgets.Button(description="Run Detection")


In [ ]:
def detect_shapes_and_edges(change):
    clear_output(wait=True)
    display(low_th, high_th, color_picker, button)

    # Values from widgets
    low = low_th.value
    high = high_th.value
    color = color_picker.value

    original = image.copy()
    gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, low, high)

    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Image 1: Detected shapes only in selected color (on black)
    shape_only_image = np.zeros_like(original)

    # Image 2: Green sketch on top of original image
    green_overlay_image = original.copy()

    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)

        # Draw filled shape only on shape_only_image
        mask = np.zeros_like(gray)
        cv2.drawContours(mask, [approx], -1, 255, -1)
        shape_only_image[mask == 255] = color

        # Add shape label
        x, y = approx.ravel()[0], approx.ravel()[1]
        if len(approx) == 3:
            shape = "Triangle"
        elif len(approx) == 4:
            shape = "Rectangle"
        elif len(approx) > 4:
            shape = "Circle"
        else:
            shape = "Unknown"
        cv2.putText(shape_only_image, shape, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # 🟩 Draw green outline on top of original image
        cv2.drawContours(green_overlay_image, [approx], -1, (0, 255, 0), 2)

    # Convert all images to RGB for matplotlib
    rgb_original = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    rgb_shapes = cv2.cvtColor(shape_only_image, cv2.COLOR_BGR2RGB)
    rgb_overlay = cv2.cvtColor(green_overlay_image, cv2.COLOR_BGR2RGB)

    # Display side-by-side
    plt.figure(figsize=(18, 6))

    plt.subplot(1, 3, 1)
    plt.imshow(rgb_original)
    plt.title("Original Image")
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(rgb_shapes)
    plt.title("Detected Shapes Only (Color: Selected)")
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(rgb_overlay)
    plt.title("Green Outline Sketch on Original")
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    # Final summary
    summary = f"""
### Final Settings Summary
**Low Threshold**: {low}
**High Threshold**: {high}
**Edge Color (Text & Fill)**: {list(color_picker.options.keys())[list(color_picker.options.values()).index(color)]}
"""
    display(Markdown(summary))


In [ ]:
# Link button click to function
button.on_click(detect_shapes_and_edges)

# Display the controls and run interface
display(low_th, high_th, color_picker, button)
